In [49]:
import re
import pandas as pd
import numpy as np

In [50]:
pd.set_option('display.max_columns', None)

In [51]:
cian_df=pd.read_csv('cian_data.csv')

D:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (33,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
cian_df=cian_df.drop_duplicates()
cian_df.reset_index(inplace=True)
cian_df=cian_df.drop('index', axis=1)

In [53]:
cian_df.describe()

,Цена,Построен,Год постройки,Подъезды,Всего комнат в квартире,Комнат в продажу
count,3.205300e+04,20513.000000,17520.000000,17201.000000,2.000000,2.000000
mean,1.507758e+07,1980.844001,1975.527797,6.288646,20.000000,1.500000
std,1.189231e+08,47.119228,46.825923,5.200089,1.414214,0.707107
min,1.199000e+06,1728.000000,1728.000000,1.000000,19.000000,1.000000
25%,5.600000e+06,1964.000000,1961.000000,3.000000,19.500000,1.250000
50%,8.000000e+06,2004.000000,1989.000000,5.000000,20.000000,1.500000
75%,1.323300e+07,2016.000000,2012.000000,8.000000,20.500000,1.750000
max,9.300000e+09,2022.000000,2020.000000,48.000000,21.000000,2.000000


In [54]:
cian_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32054 entries, 0 to 32053
Data columns (total 38 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   Название                                                                      32053 non-null  object 
 1   Описание                                                                      32053 non-null  object 
 2   Адрес                                                                         32053 non-null  object 
 3   Метро                                                                         29658 non-null  object 
 4   Время до метро                                                                29386 non-null  object 
 5   Цена                                                                          32053 non-null  float64
 6   Цена за квадрат               

In [55]:
cian_df.columns

Index(['Название', 'Описание', 'Адрес', 'Метро', 'Время до метро', 'Цена',
       'Цена за квадрат', 'Общая', 'Жилая', 'Кухня', 'Этаж', 'Построен',
       'Тип жилья', 'Планировка',
       'Площадь комнат+ обозначение смежных комнат- обозначение изолированных комнат',
       'Высота потолков', 'Санузел', 'Балкон/лоджия', 'Ремонт', 'Вид из окон',
       'Год постройки', 'Тип дома', 'Тип перекрытий', 'Подъезды', 'Лифты',
       'Отопление', 'Аварийность', 'Газоснабжение', 'Парковка', 'Мусоропровод',
       'Строительная серия', 'Срок сдачи', 'Отделка', 'Комната',
       'Всего комнат в квартире', 'Комнат в продажу', 'Площадь комнаты',
       'Размер доли'],
      dtype='object')

# Рассмотрим каждый признак, очистим данные и заполним пропущенные значения

In [56]:
#Удалим признаки, где слишком мало данных
cian_df = cian_df.drop(['Газоснабжение', 'Парковка', 'Мусоропровод',
       'Строительная серия', 'Срок сдачи', 'Отделка', 'Комната',
       'Всего комнат в квартире', 'Комнат в продажу', 'Площадь комнаты',
       'Размер доли','Планировка'], axis=1)

### Обработка признака "Общая"

In [57]:
cian_df['Общая'].isna().sum()

3

In [58]:
#Удалим пропущенные значения общей площади
cian_df=cian_df.dropna(subset=['Общая'])

In [59]:
cian_df['Общая'] = cian_df['Общая'].apply(lambda x: float(x.replace('\xa0м²','').replace('1 ','').replace(',','.').replace(' ','.')))

### Обработка признака "Жилая"

In [60]:
#Обработка пропущенных значений жилой площади:
cian_df1=cian_df.dropna(subset=['Жилая'])
cian_df1['Жилая'] = cian_df1['Жилая'].apply(lambda x: float(x.replace('\xa0м²','').replace('1 ','').replace(',','.').replace(' ','.')))
koeff=cian_df1['Общая'].mean()/cian_df1['Жилая'].mean()
cian_df['Жилая'][cian_df['Жилая'].isnull()] = cian_df['Общая'][cian_df['Жилая'].isnull()]/koeff
cian_df['Жилая'] = cian_df['Жилая'].apply(lambda x: str(x))
cian_df['Жилая'] = cian_df['Жилая'].apply(lambda x: float(x.replace('\xa0м²','').replace('1 ','').replace(',','.').replace(' ','.')))

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [61]:
cian_df['Жилая'].min()

0.0

In [62]:
#Удалим явный выброс:
cian_df['Жилая'][cian_df['Жилая']==0.0]=np.nan
cian_df = cian_df.dropna(subset=['Жилая'])

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Обработка признака "Кухня"

In [65]:
#Обработка пропущенных значений площади кухни:
cian_df2=cian_df.dropna(subset=['Кухня'])
cian_df2['Кухня'] = cian_df2['Кухня'].apply(lambda x: float(x.replace('\xa0м²','').replace('1 ','').replace(',','.').replace(' ','.')))
koeff1=cian_df2['Общая'].mean()/cian_df2['Кухня'].mean()
cian_df['Кухня'][cian_df['Кухня'].isnull()] = cian_df['Общая'][cian_df['Кухня'].isnull()]/koeff1
cian_df['Кухня'] = cian_df['Кухня'].apply(lambda x: str(x))
cian_df['Кухня'] = cian_df['Кухня'].apply(lambda x: float(x.replace('\xa0м²','').replace('1 ','').replace(',','.').replace(' ','.')))

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [66]:
cian_df['Кухня'].min()

1.0

In [67]:
cian_df['Кухня'].max()

206.0

### Обработка признака "Название"

In [68]:
cian_df['Название'] = cian_df['Название'].apply(lambda x: x.split(',')[0])

In [69]:
cian_df['Название'].value_counts()

2-комн. квартира                    8391
3-комн. квартира                    8315
1-комн. квартира                    7176
Студия                              3094
4-комн. квартира                    2473
5-комн. квартира                     758
Многокомнатная квартира              635
Апартаменты-студия                   406
1-комн. апартаменты                  304
2-комн. апартаменты                  282
3-комн. апартаменты                   87
Квартира свободной планировки         42
4-комн. апартаменты                   42
Апартаменты свободной планировки      21
Многокомнатные апартаменты            14
5-комн. апартаменты                    9
19/353 квартиры                        1
Name: Название, dtype: int64

In [70]:
cian_df['Название'][(cian_df['Название']=='Апартаменты свободной планировки')|(cian_df['Название']=='19/353 квартиры')|(cian_df['Название']=='Квартира свободной планировки')]=np.nan
cian_df=cian_df.dropna(subset=['Название'])

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [71]:
def rename(x):
    if x == 'Апартаменты-студия':
        a='Студия'
        return a
    if x == '1-комн. апартаменты':
        a='1-комн. квартира'
        return a
    if x == '2-комн. апартаменты':
        a='2-комн. квартира'
        return a 
    if x == '3-комн. апартаменты':
        a='3-комн. квартира'
        return a
    if x == '4-комн. апартаменты':
        a='4-комн. квартира'
        return a
    if x == '5-комн. апартаменты':
        a='5-комн. квартира'
        return a
    if x == 'Многокомнатные апартаменты':
        a='Многокомнатная квартира'
        return a
    else:
        return x

In [72]:
cian_df['Название'] = cian_df['Название'].apply(rename)
cian_df['Название'].value_counts()

2-комн. квартира           8673
3-комн. квартира           8402
1-комн. квартира           7480
Студия                     3500
4-комн. квартира           2515
5-комн. квартира            767
Многокомнатная квартира     649
Name: Название, dtype: int64

### Обработка признака "Цена за квадрат"

In [73]:
def replace_price_per_square(x):
    try:
        x=x.replace(' ','')
    except:
        pass
    return x

In [74]:
cian_df['Цена за квадрат'] = cian_df['Цена за квадрат'].apply(replace_price_per_square)
cian_df['Цена за квадрат']=cian_df['Цена за квадрат'].apply(lambda x:int(x))

In [75]:
#Но вообще он очень сильно коррелирует с целевой переменной, поэтому удаляем:
cian_df=cian_df.drop('Цена за квадрат', axis=1)

### Обработка признака "Адрес"

In [76]:
#Разобьём адрес на район, микрорайон, улицу. Это будет служить для дальнейшего заполнения пропусков:
cian_df['Район'] = cian_df['Адрес'].apply(lambda x: str(x.split(',')[1:2]).replace('[','').replace(']','').replace("'",''))
cian_df['Микрорайон'] = cian_df['Адрес'].apply(lambda x: str(x.split(',')[2:3]).replace('[','').replace(']','').replace("'",''))
cian_df['Улица'] = cian_df['Адрес'].apply(lambda x: str(x.split(',')[3:4]).replace('[','').replace(']','').replace("'",''))

### Обработка признака "Этаж"

In [77]:
#Создадим признак "этажей в доме":
cian_df['Этажей в доме'] = cian_df.Этаж.apply(lambda x: int(x.split(' ')[2]))

In [78]:
cian_df.Этаж = cian_df.Этаж.apply(lambda x: int(x.split(' ')[0]))

### Обработка признака "Время до метро"

#### **Переводим время до метро на транспорте в время до метро пешком**

In [79]:
#Для тех значений, где метро откроется в 2022 году, изначально поставим NaN
cian_df['Время до метро'][cian_df['Время до метро']==' откроется в 2022']=cian_df['Время до метро'][cian_df['Время до метро']==' откроется в 2022'].apply(lambda x: np.nan)

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


**Соотношение между временем на транспорте и временем пешком, на основе анализа по яндекс-картам (приблизительно):**

20 мин на транспорте - 20 мин пешком

30 тр - 45 пеш

45 тр - 90 пеш

90 тр - 280 пеш


In [80]:
#На основе этих данных написана функция по переводу минут на транспорте в минуты пешком:
def time_more_than_twenty(x):
    for i in range (1,11):    
        if x==20+i:            #если время на транспорте до метро занимает от 21 до 30 минут включительно
            x=x*(1.04**i)
    for k in range (1,16):
        if x==30+k:            #если время на транспорте до метро занимает от 31 до 45 минут включительно
            x=x*1.5*(1.02**k)
    for m in range (1,46):
        if x==45+m:            #если время на транспорте до метро занимает от 46 до 90 минут включительно
            x=x*2*(1.01**m)
    return x

In [81]:
def on_transport(x):
    if 'на транспорте' in x:
        return x

In [82]:
time_to_the_subway=cian_df['Время до метро'].dropna()

In [83]:
time_to_the_subway_on_transport=time_to_the_subway.apply(on_transport)

In [84]:
time_to_the_subway_on_transport = time_to_the_subway_on_transport.dropna()

In [85]:
time_to_the_subway_on_transport = time_to_the_subway_on_transport.apply(lambda x:int(x.replace(' мин. на транспорте','').replace(' ','').replace('⋅','').replace('<','')))

In [86]:
np.sort(time_to_the_subway_on_transport[time_to_the_subway_on_transport>20].unique())

array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
       56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 68, 70, 71, 74, 75, 82, 83,
       84, 87, 90], dtype=int64)

In [87]:
time_to_the_subway_on_transport[time_to_the_subway_on_transport>20] = time_to_the_subway_on_transport[time_to_the_subway_on_transport>20].apply(time_more_than_twenty)

In [88]:
time_to_the_subway[time_to_the_subway_on_transport.index]=time_to_the_subway_on_transport

In [89]:
time_to_the_subway = time_to_the_subway.apply(lambda x:str(x))

In [90]:
time_to_the_subway = time_to_the_subway.apply(lambda x:float(x.replace(' мин. пешком','').replace(' ','').replace('⋅','').replace('<','')))

In [91]:
cian_df['Время до метро'][time_to_the_subway.index]=time_to_the_subway

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Обработка признака "Построен"

In [92]:
cian_df['Построен'].isna().sum()

11503

In [93]:
cian_df['Год постройки'].isna().sum()

14468

In [94]:
#Может быть так, что в одной из двух колонок присутствует значение года постройки. 
#Проверим это и обоюдно дополним данные:
cian_df['Построен'][cian_df['Год постройки'].dropna().index]=cian_df['Год постройки'][cian_df['Год постройки'].dropna().index]
cian_df['Год постройки'][cian_df['Построен'].dropna().index]=cian_df['Построен'][cian_df['Построен'].dropna().index]

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [95]:
#Уберём лишний признак:
cian_df = cian_df.drop('Год постройки',axis=1)

### Обработка признака "Тип жилья" и "Построен"

In [96]:
#Год постройки в новостройках отсутствует, но на циане есть информация, что год постройки - 2022. 
#Вобьём его для пропущенных значений "Тип жилья":
cian_df['Построен'][cian_df['Тип жилья'].isnull()] = cian_df['Построен'][cian_df['Тип жилья'].isnull()].fillna(2022.0)

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [97]:
#Если год постройки = 2022, соответственно тип жилья - новостройка. Заполним пропуски:
cian_df['Тип жилья'][(cian_df['Тип жилья'].isnull())&(cian_df['Построен']==2022.0)] = cian_df['Тип жилья'][(cian_df['Тип жилья'].isnull())&(cian_df['Построен']==2022.0)].fillna('Новостройка')

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [98]:
#Оставшиеся пропуски - вторички:
cian_df['Тип жилья'][cian_df['Тип жилья'].isnull()] = cian_df['Тип жилья'][cian_df['Тип жилья'].isnull()].fillna('Вторичка')

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [99]:
def rename1(x):
    if x == 'Вторичка Апартаменты':
        a='Вторичка'
        return a
    if x == 'Вторичка Пентхаус':
        a='Вторичка'
        return a
    if x == 'Новостройка Апартаменты':
        a='Новостройка'
        return a 
    if x == 'Новостройка Пентхаус':
        a='Новостройка'
        return a
    else:
        return x

In [100]:
#Переименуем типы жилья, чтобы на выходе получилось 2 типа: Новостройка и Вторичка:
cian_df['Тип жилья'] = cian_df['Тип жилья'].apply(rename1)

In [101]:
#Заполним пропуски года постройки для новостроек:
cian_df['Построен'][cian_df['Тип жилья']=='Новостройка']=cian_df['Построен'][cian_df['Тип жилья']=='Новостройка'].fillna(2022.0)

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Обработка признака "Высота потолков"

In [102]:
cian_df['Высота потолков'][cian_df['Высота потолков'].notnull()] = cian_df['Высота потолков'][cian_df['Высота потолков'].notnull()].apply(lambda x: float(x.replace(',','.').replace(' м','')))

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [103]:
cian_df['Высота потолков'].min()

0.2

In [104]:
cian_df['Высота потолков'].max()

33.0

In [105]:
#Заменим высоту потолков в данных ниже минимальной (2,4 м) по России на эту высоту:
cian_df['Высота потолков'][cian_df['Высота потолков']<2.4] = 2.4

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [106]:
#Поставим пропущенные значения для студий, 1,2,3-х комнатных квартир, где высота по данным выше 7 метров:
cian_df['Высота потолков'][(cian_df['Высота потолков']>7.0)&(cian_df['Название']!='Многокомнатная квартира')]=np.nan

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Обработка признака "Лифты"

In [107]:
#Заполним пропуски для пропущенных значений, где количество этажей в доме =5 и меньше
cian_df['Лифты'][(cian_df['Лифты'].isnull())&(cian_df['Этажей в доме']<6)] = cian_df['Лифты'][(cian_df['Лифты'].isnull())&(cian_df['Этажей в доме']<6)].fillna('Нет')

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [108]:
cian_df.head(3)

,Название,Описание,Адрес,Метро,Время до метро,Цена,Общая,Жилая,Кухня,Этаж,Построен,Тип жилья,Площадь комнат+ обозначение смежных комнат- обозначение изолированных комнат,Высота потолков,Санузел,Балкон/лоджия,Ремонт,Вид из окон,Тип дома,Тип перекрытий,Подъезды,Лифты,Отопление,Аварийность,Район,Микрорайон,Улица,Этажей в доме
0,2-комн. квартира,Арт. 36084264 Продаётся 2-к.квартира общей пло...,"Санкт-Петербург, р-н Кировский, Дачное, Ленинс...",Автово,15,6700000.0,48.4,28.6,8.0,3,1988.0,Вторичка,12.7-15.9 м²,2.5,1 совмещенный,1 балкон,Евроремонт,На улицу и двор,Кирпичный,Железобетонные,3.0,Есть,Центральное,Нет,р-н Кировский,Дачное,Ленинский просп.,7
1,4-комн. квартира,Арт. 43292484 Продаётся 4 комнатная коммунальн...,"Санкт-Петербург, р-н Кировский, Автово, просп....",Автово,5,10900000.0,113.4,80.0,10.0,5,1956.0,Вторичка,20.2-14-21.7-24.1 м²,3,1 раздельный,NaN,Без ремонта,На улицу и двор,Кирпичный,Железобетонные,5.0,Есть,Центральное,Нет,р-н Кировский,Автово,просп. Стачек,7
2,3-комн. квартира,Арт. 43572482 В продаже просторная трёхкомнатн...,"Санкт-Петербург, р-н Кировский, Автово, просп....",Автово,8,14000000.0,103.0,63.5,20.0,3,1954.0,Вторичка,22.5-17-24 м²,3.2,1 раздельный,NaN,Косметический,На улицу и двор,Кирпичный,Железобетонные,6.0,Есть,Центральное,Нет,р-н Кировский,Автово,просп. Стачек,6


## ОБРАБОТКА ПРОПУСКОВ ПО АДРЕСУ, УЛИЦЕ, МИКРОРАЙОНУ, РАЙОНУ

In [110]:
cian_df.isna().sum()

Название                                                                            0
Описание                                                                            0
Адрес                                                                               0
Метро                                                                            2387
Время до метро                                                                   2716
Цена                                                                                0
Общая                                                                               0
Жилая                                                                               0
Кухня                                                                               0
Этаж                                                                                0
Построен                                                                         1897
Тип жилья                                             

In [112]:
#Список имён категориальных признаков:
list_categorical_features = ['Метро','Санузел','Балкон/лоджия','Тип дома','Тип перекрытий','Лифты','Отопление']

#Список имён числовых признаков:
list_numeric_features = ['Время до метро','Построен','Высота потолков','Подъезды']  

#Части адреса, которые по порядку будет проходить цикл:
parts_of_adress=['Адрес','Улица','Микрорайон','Район']                            

In [115]:
#Заполним пропущенные значения категориального признака самым частым значением в зависимости 
#от адреса, затем - от улицы, затем - от микрорайона, оставшиеся (если остались) - от района:

for feature in list_categorical_features:
    for part in parts_of_adress:
        try:
            for title in cian_df[part][cian_df[feature].isnull()].unique():
                try:
                    cian_df[feature][(cian_df[feature].isnull())&(cian_df[part]==title)]=cian_df[feature][cian_df[part]==title].value_counts().index[0]
                except:
                    pass
        except:
            pass   

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [116]:
#Заполним пропущенные значения числового признака средним значением в зависимости 
#от адреса, затем - от улицы, затем - от микрорайона, оставшиеся(если остались) - от района:

for feature in list_numeric_features:
    for part in parts_of_adress:
        try:
            for title in cian_df[part][cian_df[feature].isnull()].unique():
                try:
                    cian_df[feature][(cian_df[feature].isnull())&(cian_df[part]==title)]=cian_df[feature][cian_df[part]==title].mean()
                except:
                    pass
        except:
            pass

D:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [117]:
cian_df.isna().sum()

Название                                                                            0
Описание                                                                            0
Адрес                                                                               0
Метро                                                                               0
Время до метро                                                                      0
Цена                                                                                0
Общая                                                                               0
Жилая                                                                               0
Кухня                                                                               0
Этаж                                                                                0
Построен                                                                            0
Тип жилья                                             

## Окончательная обработка

In [118]:
#Переведём в целые значения время до метро:
cian_df['Время до метро'] = cian_df['Время до метро'].apply(lambda x: int(x))

In [119]:
#Округлим до десятых значения высоты потолков:
cian_df['Высота потолков'] = cian_df['Высота потолков'].apply(lambda x: np.round(x,1))

In [120]:
#Округлим до целого значения количество подъездов:
cian_df['Подъезды'] = cian_df['Подъезды'].apply(lambda x: np.round(x,0))

In [121]:
cian_df['Лифты'].unique()

array(['Есть', 'Нет', '2 пасс., 1 груз.', 'Есть грузовой', '7 всего',
       '14 всего', '11 всего', '2 всего', '3 пасс.', '1 пасс., 1 груз.',
       '2 пасс.', '8 всего', '6 всего', '5 всего', '3 всего', '4 всего',
       '12 всего', '30 всего', '9 всего', '3 пасс., 1 груз.', '4 пасс.',
       '2 пасс., 2 груз.', '1 пасс., 2 груз.', '16 всего', '10 всего',
       '23 всего', '4 пасс., 2 груз.', '38 всего', '18 всего', '13 всего',
       '15 всего', '2 пасс., 3 груз.', '4 пасс., 4 груз.', '46 всего',
       '4 пасс., 1 груз.', '26 всего', '3 пасс., 3 груз.', '21 всего',
       '33 всего', '25 всего', '3 пасс., 2 груз.', '19 всего', '20 всего',
       '2 груз.', '24 всего', '22 всего', '37 всего', '35 всего',
       '17 всего', '29 всего', '28 всего', '34 всего', '9 пасс.',
       '40 всего', '32 всего', '41 всего', '27 всего', '4 груз.',
       '36 всего', '44 всего', '1 пасс., 3 груз.', '31 всего', '51 всего',
       '42 всего', '53 всего', '3 груз.', '52 всего'], dtype=object)

In [122]:
#Нас по большей части интересуют именно пассажирские лифты, поэтому если они есть, переименовываем в "Есть"
#Если нет или только грузовые - "Нет":
def lifts_rename(x):
    if x == 'Есть грузовой' or x=='2 груз.' or x=='3 груз.' or x=='4 груз.':
        a='Нет'
        return a
    if x == 'Есть':
        return x
    if x == 'Нет':
        return x
    else:
        a='Есть'
        return a

In [123]:
cian_df['Лифты'] = cian_df['Лифты'].apply(lifts_rename)

In [124]:
cian_df['Лифты'].value_counts()

Есть    24351
Нет      7635
Name: Лифты, dtype: int64

### Обработка признака "Аварийность"

In [125]:
cian_df['Аварийность'].value_counts()

Нет    17890
Да         1
Name: Аварийность, dtype: int64

In [126]:
#1 дом из всех объявлений авариен. Признак бесполезен, удаляем:
cian_df = cian_df.drop('Аварийность',axis=1)

In [127]:
#Также признак не имеет полезных данных
cian_df = cian_df.drop('Площадь комнат+ обозначение смежных комнат- обозначение изолированных комнат',axis=1)

### Обработка признака "Вид из окон" и "Ремонт"

In [128]:
cian_df['Вид из окон'].value_counts()

Во двор            7564
На улицу и двор    6594
На улицу           3862
Name: Вид из окон, dtype: int64

In [129]:
cian_df['Ремонт'].value_counts()

Косметический    6746
Евроремонт       5214
Без ремонта      4868
Дизайнерский     2484
Name: Ремонт, dtype: int64

In [130]:
#Так как ремонт сугубо индивидуален, а вид из окна весьма трудно спрогнозировать, удалим их:
cian_df = cian_df.drop(['Ремонт','Вид из окон'],axis=1)

In [131]:
cian_df.isna().sum()

Название           0
Описание           0
Адрес              0
Метро              0
Время до метро     0
Цена               0
Общая              0
Жилая              0
Кухня              0
Этаж               0
Построен           0
Тип жилья          0
Высота потолков    0
Санузел            0
Балкон/лоджия      0
Тип дома           0
Тип перекрытий     0
Подъезды           0
Лифты              0
Отопление          0
Район              0
Микрорайон         0
Улица              0
Этажей в доме      0
dtype: int64

In [132]:
cian_df.head()

,Название,Описание,Адрес,Метро,Время до метро,Цена,Общая,Жилая,Кухня,Этаж,Построен,Тип жилья,Высота потолков,Санузел,Балкон/лоджия,Тип дома,Тип перекрытий,Подъезды,Лифты,Отопление,Район,Микрорайон,Улица,Этажей в доме
0,2-комн. квартира,Арт. 36084264 Продаётся 2-к.квартира общей пло...,"Санкт-Петербург, р-н Кировский, Дачное, Ленинс...",Автово,15,6700000.0,48.40,28.600000,8.000000,3,1988.0,Вторичка,2.5,1 совмещенный,1 балкон,Кирпичный,Железобетонные,3.0,Есть,Центральное,р-н Кировский,Дачное,Ленинский просп.,7
1,4-комн. квартира,Арт. 43292484 Продаётся 4 комнатная коммунальн...,"Санкт-Петербург, р-н Кировский, Автово, просп....",Автово,5,10900000.0,113.40,80.000000,10.000000,5,1956.0,Вторичка,3.0,1 раздельный,1 балкон,Кирпичный,Железобетонные,5.0,Есть,Центральное,р-н Кировский,Автово,просп. Стачек,7
2,3-комн. квартира,Арт. 43572482 В продаже просторная трёхкомнатн...,"Санкт-Петербург, р-н Кировский, Автово, просп....",Автово,8,14000000.0,103.00,63.500000,20.000000,3,1954.0,Вторичка,3.2,1 раздельный,1 балкон,Кирпичный,Железобетонные,6.0,Есть,Центральное,р-н Кировский,Автово,просп. Стачек,6
3,3-комн. квартира,Арт. 35008138 Предлагается к продаже просторна...,"Санкт-Петербург, р-н Кировский, Автово, просп....",Кировский завод,7,8900000.0,86.47,57.000000,10.000000,1,1956.0,Вторичка,3.2,1 раздельный,1 балкон,Кирпичный,Железобетонные,12.0,Есть,Центральное,р-н Кировский,Автово,просп. Стачек,10
4,Студия,"Студия в историческом центре города, 5 минуты ...","Санкт-Петербург, р-н Адмиралтейский, Сенной, Г...",Спасская,4,2500000.0,14.00,7.734003,2.587987,2,1806.0,Вторичка,3.1,1 совмещенный,1 балкон,Кирпичный,Смешанные,6.0,Нет,Центральное,р-н Адмиралтейский,Сенной,Гороховая ул.,4


In [133]:
cian_df.to_csv('cian_data_clean.csv', index=False)